In [1]:
import os
import datetime
from pathlib import Path

def collect_project_info(root_path, output_file="project_complete_info.txt"):
    """
    Thu thập cấu trúc thư mục và nội dung tất cả file code
    """
    # Các extension file code phổ biến
    code_extensions = {
        '.py', '.js', '.css', '.json', '.xml', '.yml', '.yaml',
        '.md', '.txt', '.sql', '.sh', '.bat', '.cfg', '.ini', '.env',
        '.jsx', '.ts', '.tsx', '.vue', '.php', '.java', '.cpp', '.c',
        '.h', '.cs', '.rb', '.go', '.rs', '.swift', '.kt'
    }
    
    # Thư mục và file cần bỏ qua
    ignore_dirs = {
        '__pycache__', '.git', '.vscode', '.idea', 'node_modules', 
        '.pytest_cache', '.mypy_cache', 'venv', 'env', '.env',
        'dist', 'build', '.next', '.nuxt'
    }
    
    ignore_files = {
        '.pyc', '.pyo', '.pyd', '.so', '.dll', '.exe', '.log',
        '.tmp', '.temp', '.cache', '.DS_Store', 'Thumbs.db'
    }
    
    # THÊM: Danh sách file cụ thể cần bỏ qua
    ignore_specific_files = {
        'employees.json', 'depots.json', 'transport_providers.json'
    }
    
    root_path = Path(root_path)
    
    # SỬA LỖI: Đảm bảo output_file là string
    if isinstance(output_file, Path):
        output_file = str(output_file)
    
    with open(output_file, 'w', encoding='utf-8') as f:
        # Header thông tin
        f.write("=" * 80 + "\n")
        f.write("THÔNG TIN DỰ ÁN HOÀN CHỈNH\n")
        f.write("=" * 80 + "\n")
        f.write(f"Thời gian tạo: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write(f"Thư mục gốc: {root_path.absolute()}\n")
        f.write("=" * 80 + "\n\n")
        
        # 1. Cấu trúc thư mục
        f.write("📁 CẤU TRÚC THU MỤC\n")
        f.write("-" * 50 + "\n")
        write_directory_tree(f, root_path, ignore_dirs, ignore_specific_files)
        f.write("\n\n")
        
        # 2. Danh sách tất cả file
        f.write("📄 DANH SÁCH TẤT CẢ FILE\n")
        f.write("-" * 50 + "\n")
        all_files = list_all_files(root_path, ignore_dirs, ignore_files, ignore_specific_files)
        for file_path in sorted(all_files):
            rel_path = file_path.relative_to(root_path)
            file_size = file_path.stat().st_size
            f.write(f"{rel_path} ({file_size} bytes)\n")
        f.write(f"\nTổng cộng: {len(all_files)} file\n\n")
        
        # 3. Nội dung tất cả file code
        f.write("💻 NỘI DUNG TẤT CẢ FILE CODE\n")
        f.write("=" * 80 + "\n")
        
        code_files = [f for f in all_files if f.suffix.lower() in code_extensions]
        
        for file_path in sorted(code_files):
            rel_path = file_path.relative_to(root_path)
            f.write(f"\n{'='*60}\n")
            f.write(f"FILE: {rel_path}\n")
            f.write(f"{'='*60}\n")
            
            try:
                # Thử đọc với UTF-8 trước
                content = file_path.read_text(encoding='utf-8')
                f.write(content)
            except UnicodeDecodeError:
                try:
                    # Nếu không được thì thử với encoding khác
                    content = file_path.read_text(encoding='latin-1')
                    f.write(content)
                except Exception as e:
                    f.write(f"[KHÔNG THỂ ĐỌC FILE: {e}]\n")
            except Exception as e:
                f.write(f"[LỖI ĐỌC FILE: {e}]\n")
            
            f.write(f"\n{'='*60}\n")
        
        # 4. Thống kê cuối
        f.write(f"\n\n📊 THỐNG KÊ\n")
        f.write("-" * 30 + "\n")
        f.write(f"Tổng số file: {len(all_files)}\n")
        f.write(f"File code: {len(code_files)}\n")
        f.write(f"Các loại file code:\n")
        
        extensions = {}
        for file in code_files:
            ext = file.suffix.lower()
            extensions[ext] = extensions.get(ext, 0) + 1
        
        for ext, count in sorted(extensions.items()):
            f.write(f"  {ext}: {count} file\n")

def write_directory_tree(f, path, ignore_dirs, ignore_specific_files, prefix=""):
    """Viết cấu trúc thư mục dạng tree"""
    try:
        items = sorted(path.iterdir(), key=lambda x: (x.is_file(), x.name.lower()))
        
        for i, item in enumerate(items):
            if item.name.startswith('.') and item.name not in {'.env', '.gitignore'}:
                continue
            if item.name in ignore_dirs:
                continue
            # THÊM: Bỏ qua file cụ thể
            if item.name in ignore_specific_files:
                continue
                
            is_last = i == len(items) - 1
            current_prefix = "└── " if is_last else "├── "
            
            if item.is_dir():
                f.write(f"{prefix}{current_prefix}📁 {item.name}/\n")
                next_prefix = prefix + ("    " if is_last else "│   ")
                write_directory_tree(f, item, ignore_dirs, ignore_specific_files, next_prefix)
            else:
                f.write(f"{prefix}{current_prefix}📄 {item.name}\n")
                
    except PermissionError:
        f.write(f"{prefix}[KHÔNG CÓ QUYỀN TRUY CẬP]\n")

def list_all_files(root_path, ignore_dirs, ignore_files, ignore_specific_files):
    """Lấy danh sách tất cả file"""
    all_files = []
    
    for item in root_path.rglob("*"):
        # Bỏ qua thư mục ignore
        if any(ignore_dir in item.parts for ignore_dir in ignore_dirs):
            continue
            
        # Bỏ qua file ẩn (trừ một số file quan trọng)
        if item.name.startswith('.') and item.name not in {'.env', '.gitignore', '.htaccess'}:
            continue
            
        # Bỏ qua file theo extension
        if item.suffix.lower() in ignore_files:
            continue
        
        # THÊM: Bỏ qua file cụ thể theo tên
        if item.name in ignore_specific_files:
            continue
            
        if item.is_file():
            all_files.append(item)
    
    return all_files

# Sử dụng script
if __name__ == "__main__":
    # Thay đổi đường dẫn này thành thư mục dự án của bạn
    project_path = "."  # Thư mục hiện tại
    output_file = "project_complete_info.txt"  # Đảm bảo là string
    
    print("🔄 Đang thu thập thông tin dự án...")
    collect_project_info(project_path, output_file)
    print(f"✅ Hoàn thành! Thông tin đã được lưu vào: {output_file}")
    print(f"📁 File size: {os.path.getsize(output_file)} bytes")

🔄 Đang thu thập thông tin dự án...
✅ Hoàn thành! Thông tin đã được lưu vào: project_complete_info.txt
📁 File size: 226404 bytes


In [2]:
import requests
import json

# ========================== CẤU HÌNH ==========================
# THAY THẾ URL NÀY bằng Web App URL của bạn
YOUR_WEB_APP_URL = "https://script.google.com/macros/s/AKfycbwXdhVZv5Jpxfc47kYwsk-5GcwFKYJv3cuDECudUhsuWAaEWb3eVBWoc9mVlYbVNbp1/exec"

# Secret Token phải khớp với token trong code Apps Script
SECRET_TOKEN = "atino_aPi_K3Y_2o24_sEcReT_@uTh_g00gL3"
# ===============================================================


def test_create_contract():
    """
    Gửi yêu cầu đến API để tạo một hợp đồng mới và in kết quả.
    """
    if "xxxxxxxxxxxxxxxxxxxxx" in YOUR_WEB_APP_URL:
        print("LỖI: Vui lòng thay thế giá trị của YOUR_WEB_APP_URL bằng URL thật của bạn.")
        return

    # Dữ liệu mẫu để tạo hợp đồng. 
    # Bạn có thể thay đổi các giá trị này để kiểm tra.
    payload = {
        # Token bảo mật (bắt buộc)
        "token": SECRET_TOKEN,
        
        # Các tham số bắt buộc
        "personnel_name": "Nguyễn Thị Bích Thảo",
        "date_start": "2024-08-26",          # Dùng định dạng YYYY-MM-DD
        "date_finish": "2025-08-25",
        "profile.salary_11": "12500000",
        
        # Các tham số tùy chọn khác
        "personnel_code": "ATN-0123",
        "profile.gender": "Nữ",
        "profile.birthday": "1998-10-15",
        "profile.private_code": "012345678910",
        "profile.private_code_date": "2022-05-20",
        "profile.private_code_place": "Cục CSQLHC về TTXH",
        "profile.email": "bichthao.nguyen@example.com",
        "profile.mobile": "0912345678",
        "profile.job_bank_account": "1903xxxxxxxx",
        "profile.job_bank_id": "Techcombank",
        "profile.place_current": "Số 10, đường ABC, quận Cầu Giấy, Hà Nội"
    }

    print("🚀 Đang gửi yêu cầu đến API...")
    print(f"URL: {YOUR_WEB_APP_URL}")
    print("--------------------------------------------------")

    try:
        # Gửi yêu cầu GET với các tham số
        response = requests.get(YOUR_WEB_APP_URL, params=payload, timeout=30)
        
        # Kiểm tra mã trạng thái HTTP
        response.raise_for_status()  # Sẽ báo lỗi nếu mã trạng thái là 4xx hoặc 5xx

        # Phân tích kết quả JSON
        result = response.json()

        print("✅ Yêu cầu thành công!")
        print("--------------------------------------------------")
        
        # In kết quả một cách đẹp mắt
        print(json.dumps(result, indent=2, ensure_ascii=False))

        if result.get("status") == "success":
            print("\n🎉 Hợp đồng đã được tạo thành công!")
            print(f"🔗 Link Google Doc: {result['data']['docUrl']}")
            print(f"📄 Link PDF: {result['data']['pdfUrl']}")
        else:
            print(f"\n❌ API trả về lỗi: {result.get('message')}")

    except requests.exceptions.HTTPError as http_err:
        print(f"❌ Lỗi HTTP: {http_err}")
        print(f"Nội dung phản hồi: {response.text}")
    except requests.exceptions.RequestException as err:
        print(f"❌ Lỗi kết nối: {err}")
    except json.JSONDecodeError:
        print("❌ Không thể phân tích phản hồi JSON. Phản hồi từ server không hợp lệ.")
        print(f"Nội dung phản hồi thô: {response.text}")


if __name__ == "__main__":
    test_create_contract()

🚀 Đang gửi yêu cầu đến API...
URL: https://script.google.com/macros/s/AKfycbwXdhVZv5Jpxfc47kYwsk-5GcwFKYJv3cuDECudUhsuWAaEWb3eVBWoc9mVlYbVNbp1/exec
--------------------------------------------------
✅ Yêu cầu thành công!
--------------------------------------------------
{
  "status": "success",
  "message": "Contract created successfully",
  "data": {
    "docUrl": "https://docs.google.com/open?id=1nFgg-Hyi2IjCNVzotnYCrlg7ADq9oLB05635DIv5LKU",
    "pdfUrl": "https://drive.google.com/file/d/18QlGEo-XDQxSK1uWVjBZUs_Xy06mFSDQ/view?usp=drivesdk",
    "fileName": "HĐ Khoán Việc - Nguyễn Thị Bích Thảo - 26/08/2024"
  }
}

🎉 Hợp đồng đã được tạo thành công!
🔗 Link Google Doc: https://docs.google.com/open?id=1nFgg-Hyi2IjCNVzotnYCrlg7ADq9oLB05635DIv5LKU
📄 Link PDF: https://drive.google.com/file/d/18QlGEo-XDQxSK1uWVjBZUs_Xy06mFSDQ/view?usp=drivesdk


In [3]:
import requests
import json

# ==============================================================================
# CẤU HÌNH CẦN THAY ĐỔI
# ==============================================================================

# !!! QUAN TRỌNG: Dán URL ứng dụng web của bạn vào đây.
# URL này bạn nhận được sau khi triển khai (deploy) Google Apps Script.
WEB_APP_URL = "https://script.google.com/macros/s/AKfycbztyeMTcRZq5qcitpIzN6Azi2KCKfCwW89Y2IlxMIGHYyIPjazMrea9k5o1lWJ68JsT/exec"

# Mã bí mật này phải khớp với mã trong Google Apps Script
SECRET_TOKEN = "atino_aPi_K3Y_2o24_sEcReT_@uTh_g00gL3"


# ==============================================================================
# DỮ LIỆU MẪU ĐỂ GỬI ĐI
# ==============================================================================

# Đây là dữ liệu mẫu để điền vào hợp đồng thử việc.
# Bạn có thể thay đổi các giá trị này để kiểm tra.
test_data = {
    "token": SECRET_TOKEN,

    # Thông tin hợp đồng (Tùy chọn, nếu bỏ trống script sẽ tự tạo)
    "code": "HDTV-PY-TEST-001",

    # Thông tin nhân viên
    "personnel_name": "Nguyễn Văn A (Test)",
    "personnel_code": "NV0099",
    "department_id": "Phòng Kinh Doanh",

    # Thông tin chi tiết (profile)
    "profile.gender": "Nam",
    "profile.birthday": "1995-10-20",  # Định dạng YYYY-MM-DD
    "profile.level_academic": "Đại học",
    "profile.private_code": "012345678912",
    "profile.private_code_date": "2020-07-15", # Định dạng YYYY-MM-DD
    "profile.private_code_place": "Cục Cảnh sát QLHC về TTXH",
    "profile.job_bank_account": "102003004005",
    "profile.job_bank_id": "Vietinbank",
    "profile.place_home": "Số 10, đường ABC, phường XYZ, quận GHK, Hà Nội",
    "profile.place_current": "Số 25, đường DEF, phường UVW, quận LMN, Hà Nội",
    "profile.mobile": "0912345678",
    "profile.email": "nguyenvana.test@email.com",
    "profile.position_id": "Nhân viên Bán hàng",
    "profile.job_date_join": "2025-08-26", # Định dạng YYYY-MM-DD
    "profile.job_date_end_review": "2025-10-25", # Định dạng YYYY-MM-DD

    # Thông tin lương
    "profile.salary_11": "9500000", # Nhập số liền, không có dấu chấm phẩy
}


# ==============================================================================
# HÀM GỬI YÊU CẦU
# ==============================================================================

def test_create_contract():
    """
    Hàm gửi yêu cầu GET đến Google Apps Script và in ra kết quả.
    """
    print("Đang gửi yêu cầu đến URL:", WEB_APP_URL)
    print("Với các tham số:")
    print(json.dumps(test_data, indent=2, ensure_ascii=False))
    print("-" * 30)

    try:
        # Gửi yêu cầu GET với các tham số đã định nghĩa
        response = requests.get(WEB_APP_URL, params=test_data)

        # Kiểm tra xem có lỗi HTTP không (ví dụ: 404, 500)
        response.raise_for_status()

        # Chuyển đổi phản hồi từ dạng text sang JSON
        result = response.json()

        print("✅ Yêu cầu thành công! Phản hồi từ server:")
        # In kết quả JSON ra một cách dễ đọc
        print(json.dumps(result, indent=4, ensure_ascii=False))

        # Nếu thành công, in ra các đường link
        if result.get("status") == "success":
            print("\n---")
            print(f"File hợp đồng đã được tạo: {result['data']['fileName']}")
            print(f"🔗 Link Google Doc: {result['data']['docUrl']}")
            print(f"🔗 Link PDF: {result['data']['pdfUrl']}")
            print("---\n")
            print("Hãy kiểm tra thư mục Google Drive của bạn!")


    except requests.exceptions.HTTPError as http_err:
        print(f"❌ Lỗi HTTP xảy ra: {http_err}")
        print(f"Nội dung phản hồi lỗi: {response.text}")
    except requests.exceptions.RequestException as err:
        print(f"❌ Lỗi kết nối hoặc yêu cầu: {err}")
    except json.JSONDecodeError:
        print("❌ Không thể phân tích phản hồi JSON. Phản hồi thô từ server:")
        print(response.text)

# Chạy hàm test
if __name__ == "__main__":
    if "YOUR_WEB_APP_ID" in WEB_APP_URL:
        print("!!! CẢNH BÁO: Bạn chưa thay thế `WEB_APP_URL` bằng URL thật.")
    else:
        test_create_contract()

Đang gửi yêu cầu đến URL: https://script.google.com/macros/s/AKfycbztyeMTcRZq5qcitpIzN6Azi2KCKfCwW89Y2IlxMIGHYyIPjazMrea9k5o1lWJ68JsT/exec
Với các tham số:
{
  "token": "atino_aPi_K3Y_2o24_sEcReT_@uTh_g00gL3",
  "code": "HDTV-PY-TEST-001",
  "personnel_name": "Nguyễn Văn A (Test)",
  "personnel_code": "NV0099",
  "department_id": "Phòng Kinh Doanh",
  "profile.gender": "Nam",
  "profile.birthday": "1995-10-20",
  "profile.level_academic": "Đại học",
  "profile.private_code": "012345678912",
  "profile.private_code_date": "2020-07-15",
  "profile.private_code_place": "Cục Cảnh sát QLHC về TTXH",
  "profile.job_bank_account": "102003004005",
  "profile.job_bank_id": "Vietinbank",
  "profile.place_home": "Số 10, đường ABC, phường XYZ, quận GHK, Hà Nội",
  "profile.place_current": "Số 25, đường DEF, phường UVW, quận LMN, Hà Nội",
  "profile.mobile": "0912345678",
  "profile.email": "nguyenvana.test@email.com",
  "profile.position_id": "Nhân viên Bán hàng",
  "profile.job_date_join": "2025-

In [4]:
import requests
import json
from pprint import pprint  # Dùng để in ra JSON cho đẹp mắt

def test_fetch_imex_details(bill_id: int):
    """
    Hàm này gửi yêu cầu đến API Nhanh.vn để lấy thông tin chi tiết
    của một Bill ID cụ thể và in kết quả ra màn hình.
    """
    print(f"--- Đang lấy thông tin cho Bill ID: {bill_id} ---")

    # URL của API Nhanh.vn, được lấy từ file src/utils/api.py của bạn
    url = "https://open.nhanh.vn/api/bill/imexrequirements"

    # Dữ liệu gửi đi (payload), bao gồm các thông tin xác thực
    # và Bill ID cần tra cứu.
    payload = {
        "version": "2.0",
        "appId": "74951",
        "businessId": "8901",
        "accessToken": "twf9P1xFZCUUgwt8zR0XgNeB6V5jsbq2KHb14bxovqK1ppCxyADwOK8FzQlCEeEGABRZINXoUCSzM50kjhwcrUSBWTY5nSvyhfnH2X2cI0pC7pNczSVxc1ratdDmxF85q7hUTUNCrUnpPTG5ZwLNO7bkMlEEJTCdPhgYaC",
        "data": json.dumps({"billId": bill_id})  # Dữ liệu phải là một chuỗi JSON
    }

    try:
        # Gửi yêu cầu POST đến API với timeout là 20 giây
        response = requests.post(url, data=payload, timeout=20)

        # Kiểm tra xem yêu cầu có thành công về mặt HTTP không (status code 200)
        if response.status_code == 200:
            print("✅ Yêu cầu HTTP thành công (Status Code 200).")
            
            # Chuyển đổi nội dung phản hồi từ chuỗi JSON sang đối tượng Python
            response_json = response.json()

            # API Nhanh.vn trả về 'code: 1' nếu logic thành công
            if response_json.get("code") == 1:
                print("✅ API Nhanh.vn xử lý thành công (API Code 1).")
                
                data = response_json.get("data", {})
                imexs = data.get("imexs")

                if imexs:
                    print("\n--- Dữ liệu chi tiết nhận được ---")
                    pprint(imexs)
                else:
                    print("\n⚠️ API trả về thành công nhưng không có dữ liệu 'imexs' cho Bill ID này.")
            
            else:
                # Trường hợp API Nhanh.vn trả về lỗi (ví dụ: Bill ID không tồn tại)
                print("\n❌ Lỗi từ API Nhanh.vn:")
                pprint(response_json)

        else:
            # Trường hợp lỗi HTTP (ví dụ: 404 Not Found, 500 Internal Server Error)
            print(f"\n❌ Yêu cầu HTTP thất bại. Status Code: {response.status_code}")
            print("Nội dung lỗi:", response.text)

    except requests.exceptions.RequestException as e:
        # Trường hợp lỗi kết nối mạng, timeout, etc.
        print(f"\n❌ Đã xảy ra lỗi kết nối: {e}")
    except json.JSONDecodeError:
        # Trường hợp phản hồi không phải là JSON hợp lệ
        print("\n❌ Lỗi: Không thể phân tích dữ liệu JSON từ phản hồi của API.")
        print("Nội dung phản hồi:", response.text)


# --- Chạy chương trình ---
if __name__ == "__main__":
    # Thay đổi ID ở đây nếu bạn muốn test với các ID khác
    id_can_test = 7765739
    
    test_fetch_imex_details(id_can_test)

--- Đang lấy thông tin cho Bill ID: 7765739 ---
✅ Yêu cầu HTTP thành công (Status Code 200).
✅ API Nhanh.vn xử lý thành công (API Code 1).

--- Dữ liệu chi tiết nhận được ---
{'68253024': {'approvedAt': '1756115039',
              'approvedById': '3504395',
              'approvedByUser': 'AM.0883 - Lê Trung Kiên',
              'approvedDescription': None,
              'approvedPrice': '41008',
              'approvedQuantity': '10',
              'billId': '7765739',
              'confirmedAt': None,
              'damagedQuantity': None,
              'discount': None,
              'fromDepotId': '170507',
              'fromDepotName': 'KHO CỬA HÀNG',
              'id': '68253024',
              'imeiId': None,
              'mode': '3',
              'modeName': '[C]',
              'productBatchId': None,
              'productId': '37927235',
              'productStore': {'barcode': '2000214899610',
                               'code': '1627.10012.V91.QS.VN.2025.4.1.100% 